In [1]:
!pip install -q mediapipe==0.10.0

zsh:1: command not found: pip


For my MacBook lol

In [ ]:
!python3 -m pip install -q mediapipe==0.10.0

**Code for detecting color blue through Intel RealSense D455 Camera (livestream)**

In [ ]:
import cv2 as cv
import numpy as np
import cv2
vid = cv2.VideoCapture(0)
while(1):
    # Take each frame
    _, frame = vid.read()
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # define range of blue color in HSV
    lower_blue = np.array([80,160,50]) # The RealSense Camera HSV values are different than the norm
    upper_blue = np.array([130,255,255])
    # Threshold the HSV image to get only blue colors
    mask = cv.inRange(hsv, lower_blue, upper_blue)
    # Bitwise-AND mask and original image
    res = cv.bitwise_and(frame,frame, mask= mask)
    cv.imshow('frame',frame)
    cv.imshow('mask',mask)
    cv.imshow('res',res)
    if cv2.waitKey(5) & 0xFF ==ord("q"):
        break
cv.destroyAllWindows()

**Face detection code**

In [ ]:
import cv2
import mediapipe as mp

#Variables
face_detection=mp.solutions.face_detection.FaceDetection()
drawing_utils=mp.solutions.drawing_utils

vid = cv2.VideoCapture(0) #Change value according to input

while vid.isOpened():
    success,frame = vid.read()
    #Face detection
    frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) #convert to rgb for mp
    endframe=face_detection.process(frame)
    #Display
    frame=cv2.cvtColor(frame,cv2.COLOR_RGB2BGR) #convert back to bgr for cv
    if endframe.detections:
        for detection in endframe.detections:
            drawing_utils.draw_detection(frame,detection)
    cv2.imshow("Webcam",frame)
    if cv2.waitKey(5) & 0xFF ==ord("q"):
        break

**Object detection code**

In [2]:
!wget -q -O efficientdet.tflite -q https://storage.googleapis.com/mediapipe-models/object_detector/efficientdet_lite0/int8/1/efficientdet_lite0.tflite

import cv2
import mediapipe as mp
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

MARGIN = 10  # pixels
ROW_SIZE = 10  # pixels
FONT_SIZE = 2
FONT_THICKNESS = 1
TEXT_COLOR = (255, 0, 0)  # red

#Capture webcam video according to input
vid = cv2.VideoCapture(0)

while vid.isOpened():
    success,frame = vid.read()
    
    #Variables
    VisionRunningMode = mp.tasks.vision.RunningMode
    base_options = python.BaseOptions(model_asset_path='efficientdet.tflite')
    options = vision.ObjectDetectorOptions(base_options=base_options,
                                       score_threshold=0.5)
    detector = vision.ObjectDetector.create_from_options(options)
    
    def visualize(
        frame,
        detection_result
    ) -> np.ndarray:
    
      for detection in detection_result.detections:
        # Draw bounding_box
        bbox = detection.bounding_box
        start_point = bbox.origin_x, bbox.origin_y
        end_point = bbox.origin_x + bbox.width, bbox.origin_y + bbox.height
        cv2.rectangle(frame, start_point, end_point, TEXT_COLOR, 3)
        # Draw label and score
        category = detection.categories[0]
        category_name = category.category_name
        probability = round(category.score, 2)
        result_text = category_name + ' (' + str(probability) + ')'
        text_location = (MARGIN + bbox.origin_x,
                         MARGIN + ROW_SIZE + bbox.origin_y)
        cv2.putText(frame, result_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                    FONT_SIZE, TEXT_COLOR, FONT_THICKNESS)

      return frame

    frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    frame = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
    detection_result = detector.detect(frame)
    image_copy = np.copy(frame.numpy_view())
    annotated_frame = visualize(image_copy, detection_result)
    frame = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
    cv2.imshow("Webcam",frame)
    if cv2.waitKey(5) & 0xFF ==ord("q"):
        break
        #yeehaw